## creating a mongodb database and collection

In [6]:
import pymongo
from pymongo import MongoClient

# connect to MongoDB
client = MongoClient("mongodb://localhost:27017/")

db = client['gym_membership']
collection = db['members']

collection.insert_one({'name': 'sai', 'age': 20})

print("data inserted successfully")

data inserted successfully


## data inserting using CSV file

In [8]:
import pandas as pd
from pymongo import MongoClient

# Step 1: Read the Excel file
csv_file_path = 'gym_membership.csv'
df = pd.read_csv(csv_file_path)

# Display the first few rows of the DataFrame
print("Data from csv file:")
print(df.head())

# Step 2: Connect to MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["gym_membership"]
collection = db["members"]

# Step 3: Convert DataFrame to a list of dictionaries
data = df.to_dict(orient='records')

# Step 4: Insert data into MongoDB collection
result = collection.insert_many(data)

# Output the IDs of the inserted documents
print("Inserted document IDs:", result.inserted_ids)


Data from csv file:
   id  gender    birthday  Age abonoment_type  visit_per_week  \
0   1  Female  1997-04-18   27        Premium               4   
1   2  Female  1977-09-18   47       Standard               3   
2   3    Male  1983-03-30   41        Premium               1   
3   4    Male  1980-04-12   44        Premium               3   
4   5    Male  1980-09-10   44       Standard               2   

        days_per_week  attend_group_lesson            fav_group_lesson  \
0  Mon, Sat, Tue, Wed                 True  Kickboxen, BodyPump, Zumba   
1       Mon, Sat, Wed                False                         NaN   
2                 Sat                 True                       XCore   
3       Sat, Tue, Wed                False                         NaN   
4            Thu, Wed                 True        Running, Yoga, Zumba   

  avg_time_check_in avg_time_check_out  avg_time_in_gym  drink_abo  \
0          19:31:00           21:27:00              116      False   
1   

In [17]:
from pymongo import MongoClient

client = MongoClient('mongodb://localhost:27017/')
db = client['gym_membership']

db.create_collection('memberss')

print('collection created successfully')

collection created successfully


## transfering a data from collection to collection inside the database

In [28]:
from pymongo import MongoClient

client = MongoClient('mongodb://localhost:27017/')
db = client['gym_membership']

# access the collections in the database
source_collection = db['members']
target_collection = db['memberss']

# count all the documents in he source collection
total_documents = source_collection.count_documents({})
print(f"Total documents in the source collection: {total_documents}")

# calculate half the documents
half_count = total_documents // 2
print(f"Number of documents to transfer: {half_count}")

if half_count == 0:
    print("No documents to transfer.")
else:
    # Step 5: Fetch half of the documents from the source collection
    documents_to_transfer = source_collection.find().limit(half_count)
    documents_list = list(documents_to_transfer)  # Convert cursor to list
    
    # Step 6: Insert the documents into the target collection
    if documents_list:
        target_collection.insert_many(documents_list)
        print(f"Transferred {len(documents_list)} documents to target collection.")
        
        # Step 7 (Optional): Delete the transferred documents from the source collection
        
        # Collect the _id values of the transferred documents
        
        transferred_ids = [doc['_id'] for doc in documents_list]
        source_collection.delete_many({'_id': {'$in': transferred_ids}})
        print(f"Deleted {len(transferred_ids)} documents from source collection.")
        
        # Step 8: Verify the collections
        
        print(f"Documents remaining in source collection: {source_collection.count_documents({})}")
        print(f"Documents in target collection: {target_collection.count_documents({})}")
        
    # Step 9: Close the connection
client.close()


Total documents in the source collection: 1003
Number of documents to transfer: 501
Transferred 501 documents to target collection.
Deleted 501 documents from source collection.
Documents remaining in source collection: 502
Documents in target collection: 501


## copy the data from source database to target(new) database

In [5]:
from pymongo import MongoClient

# Connect to MongoDB properly
client = MongoClient('mongodb://localhost:27017/')

# Specify the source and target databases
source_db = client['gym_membership']  # Access source database
target_db = client['gym_membership_backup']  # Create/access target database

# Get all collections in the source database
collections = source_db.list_collection_names()

for collection_name in collections:
    source_collection = source_db[collection_name]
    target_collection = target_db[collection_name]

    # Fetch all documents from the source collection
    documents = list(source_collection.find())

    if documents:
        target_collection.insert_many(documents)
        print(f"Copied {len(documents)} documents from {collection_name} to the new database.")

print("Data copy process completed successfully!")

# Close the MongoDB connection
client.close()


Copied 502 documents from members to the new database.
Copied 501 documents from memberss to the new database.
Data copy process completed successfully!
